# Import packages

In [1]:
using LinearAlgebra, MAT, DelimitedFiles

# Auxiliary Functions

In [2]:
function ldet(A)
    (value,sign_det) = logabsdet(A);
    if (sign_det > 0)
        return value
    else
        return -Inf
    end
end

function ldet_objval(A,x)
    return ldet(A'*diagm(vec(x))*A);
end

ldet_objval (generic function with 1 method)

In [3]:
function getMatlabInstance(instanceName,matrixName)
    myInst = string(".\\Instances\\",instanceName,".mat");
    file = matopen(string(myInst));
    A = read(file, string(matrixName)); # note that this does NOT introduce a variable ``varname`` into scope
    close(file);
    return A
end

function saveMatlabInstance(instanceName,matrixName,matrix)
    if !isdir(".\\ResultsInstances")
        mkdir(".\\ResultsInstances")
    end
    myInst = string(".\\ResultsInstances\\",instanceName,".mat");
    matwrite(myInst,Dict(matrixName => matrix));
    return
end

function writeTXT(out_name,notes)
    if !isdir(".\\ResultsTXT")
        mkdir(".\\ResultsTXT")
    end
    outfile = string(".\\ResultsTXT\\",out_name);
    f = open(outfile, "w");
    for i in eachindex(notes)
        println(f, notes[i]);
    end
    close(f);
end

writeTXT (generic function with 1 method)

# Local Search Functions

## Initial Solutions

In [4]:
function initBinary(A,R,s,m,n)
    F = svd(A,full= true);
    U = F.U;
    x = zeros(n,1);
    for j = (1:n)
       for i = (1:s) 
           x[j] = x[j] + U[j,i]^2; 
       end
    end
    x_save = copy(x);
    x = zeros(n,1);
    for row in R
        x[row] = 1;
        x_save[row] = 0;
    end
    for i = (1:s-m)
        max_indice = argmax(x_save);
        x[max_indice] = 1;
        x_save[max_indice] = 0;
    end
    zlb = ldet_objval(A,x);
    xlb = x;  
    return xlb,zlb
end

function initGreedy(A,R,s,m,n)
    F = svd(A,full= true);
    U = F.U;
    S = F.S;
    x = zeros(n,1);
    k = min(s,m);
    for j = (1:n)
       for i = (1:k)
           x[j] = x[j] + (S[i]*U[j,i])^2; 
       end
    end
    x_save = copy(x);
    x = zeros(n,1);
    for row in R
        x[row] = 1;
        x_save[row] = 0;
    end
    for i = (1:s-m)
        max_indice = argmax(x_save);
        x[max_indice] = 1;
        x_save[max_indice] = 0;
    end
    zlb = ldet_objval(A,x);
    xlb = x;  
    return xlb,zlb
end

initGreedy (generic function with 1 method)

## Local Searches

In [5]:
function LSFI(A,n,x_init,z_lb) # Local Search First Improvement
    x = copy(x_init);
    flag = true;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            z_lb = z_lb_new;
                            flag = true;
                            break 
                        else
                            x[j] = 0;
                        end
                    end
                end
                if flag
                    break 
                else
                    x[i] = 1;
                end
            end
        end
    end
    return x,z_lb
end


function LSFP(A,n,x_init,z_lb) # Local Search First Improvement Plus
    x = copy(x_init);
    flag = true;
    leave_x,enter_x = 0,0;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            leave_x = i;
                            enter_x = j;
                            z_lb = z_lb_new;
                            flag = true; 
                        end
                        x[j] = 0;
                    end
                end
                if flag
                    break 
                else
                    x[i] = 1;
                end
            end
        end
        if flag
            #x[leave_x] = 0;
            x[enter_x] = 1;
        end
    end
    return x,z_lb
end

function LSBI(A,n,x_init,z_lb) # Local Search First Improvement Plus
    x = copy(x_init);
    flag = true;
    leave_x,enter_x = 0,0;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            leave_x = i;
                            enter_x = j;
                            z_lb = z_lb_new;
                            flag = true; 
                        end
                        x[j] = 0;
                    end
                end
                x[i] = 1;
            end
        end
        if flag
            x[leave_x] = 0;
            x[enter_x] = 1;
        end
    end
    return x,z_lb
end

LSBI (generic function with 1 method)

In [7]:
function runLocalSearch(A,R,n,m,s)
    # Initial solution
    t_init_bin = @elapsed x_init_bin,z_init_bin = initBinary(A,R,s,m,n);
    t_init_gdy = @elapsed x_init_gdy,z_init_gdy = initGreedy(A,R,s,m,n);
    # Get initial results
    X = [x_init_bin,x_init_gdy];
    Z = [z_init_bin,z_init_gdy];
    T = [t_init_bin,t_init_gdy];
    # Heuristics
    X_init = [x_init_bin,x_init_gdy];
    for x_init in X_init
        z_init = ldet_objval(A,x_init);
        t_FI   = @elapsed x_FI, z_FI = LSFI(A,n,x_init,z_init);
        t_FP   = @elapsed x_FP, z_FP = LSFP(A,n,x_init,z_init);
        t_BI   = @elapsed x_BI, z_BI = LSBI(A,n,x_init,z_init);
        push!(X,x_FI,x_FP,x_BI);
        push!(Z,z_FI,z_FP,z_BI);
        push!(T,t_FI,t_FP,t_BI);
    end
    z_heur = maximum(Z);
    indsX  = findall(Z .== z_heur); # we can receive more than one indice here
    x_heur = X[indsX[1]];
    # More informations
    sum_x = [];
    max_x = [];
    for el in X
        push!(sum_x,sum(el));
        push!(max_x,maximum(el));
    end
    # All Informations
    infos = (Z,T,sum_x,max_x,indsX);
    return x_heur, z_heur,infos
end

runLocalSearch (generic function with 1 method)

# Run code

In [9]:
function getMatlabInitialInfos(n,i)
    nameInst = string("Instance_",n,"_",i);
    A = getMatlabInstance(nameInst,"A");
    R = trunc.(Int,getMatlabInstance(nameInst,"R"));
    s = trunc(Int,n/2);
    m = trunc(Int,n/4);
    return A,R,m,s
end

getMatlabInitialInfos (generic function with 1 method)

In [10]:
function writeAllInfos(Results,InfoLS,info_ls,results)
    if !isdir(".\\ResultsCSV")
        mkdir(".\\ResultsCSV")
    end
    # File name
    resultsstr = string(".\\ResultsCSV\\results.csv");
    zLSstr     = string(".\\ResultsCSV\\zLS.csv");
    tLSstr     = string(".\\ResultsCSV\\tLS.csv");
    sum_str    = string(".\\ResultsCSV\\sumLS.csv");
    max_str    = string(".\\ResultsCSV\\maxLS.csv");
    # Get infos
    z       = info_ls[1];
    t       = info_ls[2];
    sum_x   = info_ls[3];
    max_x   = info_ls[4];
    # push new informations
    push!(Results,results);
    push!(InfoLS[1],z);
    push!(InfoLS[2],t);
    push!(InfoLS[3],sum_x);
    push!(InfoLS[4],max_x);
    # save results
    writedlm(resultsstr,  Results, ',')
    writedlm(zLSstr,InfoLS[1], ',');
    writedlm(tLSstr,InfoLS[2], ',');
    writedlm(sum_str,InfoLS[3], ',');
    writedlm(max_str,InfoLS[4], ',');
    return Results,InfoLS
end

writeAllInfos (generic function with 1 method)

In [12]:
function saveFiles(n,i,x_ls)
    x_ls_name   = string("x_ls_",n,"_",i);
    saveMatlabInstance(x_ls_name,"x_ls",x_ls);
    return
end

saveFiles (generic function with 1 method)

In [ ]:
Results = [];
InfoLS  = [[],[],[],[]];
InfoLS_old  = [[],[],[],[]];
my_N = [40,60,80,100,140,180,200,240,280,300];
for n in my_N
    for i in (1:3)
        A,R,m,s = getMatlabInitialInfos(n,i);
        time_ls = @elapsed x_ls, z_ls,info_ls = runLocalSearch(A,R,n,m,s);
        println("$(n),$(i): Finished LS $(z_ls) with $(time_ls) sec")
        flush(stdout); # show println
        results  = [n,m,s,i,z_ls,time_ls];
        Results,InfoLS = writeAllInfos(Results,InfoLS,info_ls,results);
        saveFiles(n,i,x_ls)
    end
end

40,1: Finished LS -1.5538065375157495 with 2.310842872 sec
40,2: Finished LS -0.7013141749532329 with 0.559851382 sec
40,3: Finished LS -1.4265686998844604 with 0.169027267 sec
60,1: Finished LS -1.7811835092404427 with 2.630542684 sec
60,2: Finished LS -2.3348248346041665 with 1.864293548 sec
60,3: Finished LS -1.40011221194216 with 2.475871781 sec
80,1: Finished LS -2.7777165416793443 with 30.280470951 sec
80,2: Finished LS -2.4262601319857082 with 29.260218085 sec
80,3: Finished LS -2.487376083484186 with 29.44003219 sec
100,1: Finished LS -2.426174239847802 with 90.36396591 sec
100,2: Finished LS -2.7135833838735475 with 106.593506728 sec
100,3: Finished LS -2.6575468031818037 with 86.821771959 sec
140,1: Finished LS -3.0773270100606767 with 529.859060018 sec
140,2: Finished LS -3.2177118307946273 with 488.533374217 sec
140,3: Finished LS -1.5186079694057744 with 524.626760827 sec
180,1: Finished LS -2.0571155641648318 with 1750.326732378 sec
180,2: Finished LS -1.6546466910029753 